### **Importing Related Notebooks** 

In [2]:
import import_ipynb
import Model_Benchmarking

gb = Model_Benchmarking.gb
X_trainval = Model_Benchmarking.X_trainval
y_trainval = Model_Benchmarking.y_trainval
X_test = Model_Benchmarking.X_test
y_test = Model_Benchmarking.y_test
skfold = Model_Benchmarking.skfold

### **Imbalanced Dataset**

As previously known, the dataset as a whole has an unequal proportion of target classes where in this case study, the number of customers who churn is far less than subscribers who are still subscribed. This can cause several disadvantages, including model performance that tends to be biased, poor generalization, non-representative measurement evaluation results, and wrong interpretation of existing problems. This happens because the amount of data for the minority class, in this case churn customers, is insufficient when the training process is carried out on the algorithm model used. As a result, the algorithm model will produce poor performance in detecting churn customers even though overall it has fairly good accuracy. These deficiencies can be anticipated by manipulating the data so that the number of positive and negative classes is equal.

##### **Imbalanced Cross Validation Score**

The strategy that can be done is to use several class that are already available by **imblearn**, including `RandomOverSampler()`, `SMOTE()`, `RandomUnderSampler()`, and ` NearMiss()`. Each class has its own advantages and disadvantages in handling imbalanced data cases so a cross validation score process will be carried out to find the average performance of each class against the dataset used for the training process and validation, namely `X_trainval` and `y_trainval`. Then, a testing process will be carried out on dataset `X_test` and `y_test`.

This will be easier by creating an artificial class that is capable of carrying out the process. This class consists of three methods which have their respective designations, namely `imbalance_validation()`, `imbalance_test_score()`, and `imbalance_classification_report()`.

In [3]:
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from sklearn.metrics import fbeta_score, average_precision_score, precision_score, recall_score, classification_report

class imbalance_val_score:
    def __init__(self,estimator):
        self.estimator = estimator
        self.methods = ['1','2','3','4','5']
    
    def imbalance_validation(self,X_trainval,y_trainval):
        ap_scores, f2_scores, pr_scores, re_scores = [], [], [], []
        result_scores = [ap_scores, f2_scores, pr_scores, re_scores]
        
        for method in self.methods:
            ap_method, f2_method, pr_method, re_method = [], [], [], []
            method_scores = [ap_method, f2_method, pr_method, re_method]
            
            for train, validation in skfold.split(X_trainval,y_trainval):
                X_train = X_trainval.iloc[train]
                y_train = y_trainval.iloc[train]
                X_val = X_trainval.iloc[validation]
                y_val = y_trainval.iloc[validation]
                scores = (self.method(
                    estimator=self.estimator,
                    X_train=X_train,
                    y_train=y_train,
                    X_test=X_val,
                    y_test=y_val,
                    method=method
                ).method_score())
                
                for i in zip(method_scores,scores):
                    i[0].append(i[1])
            
            for i in zip(result_scores,method_scores):
                i[0].append(i[1])
        
        return ap_scores, f2_scores, pr_scores, re_scores
    
    def imbalance_test_score(self,X_trainval,y_trainval,X_test,y_test):
        ap_scores, f2_scores, pr_scores, re_scores = [], [], [], []
        result_scores = [ap_scores, f2_scores, pr_scores, re_scores]
        
        for method in self.methods:
            scores = (self.method(
                estimator=self.estimator,
                X_train=X_trainval,
                y_train=y_trainval,
                X_test=X_test,
                y_test=y_test,
                method=method
            ).method_score())
            
            for i in zip(result_scores,scores):
                i[0].append(i[1])
        
        return ap_scores, f2_scores, pr_scores, re_scores
    
    def imbalance_classification_report(self,X_trainval,y_trainval,X_test,y_test):
        reports = []

        for method in self.methods:
            reports.append(self.method(
                estimator=self.estimator,
                X_train=X_trainval,
                y_train=y_trainval,
                X_test=X_test,
                y_test=y_test,
                method=method
            ).method_report())
        
        return reports

    class method:
        def __init__(self,estimator,X_train,y_train,X_test,y_test,method):
            self.X_train = X_train
            self.y_train = y_train
            self.X_test = X_test
            self.y_test = y_test
            self.estimator = estimator 
            self.method = method

        def method_score(self):
            X_resampled, y_resampled = self.selection(
                X_train=self.X_train,
                y_train=self.y_train,
                method=self.method
            ).get_samples()

            scores = (self.calculate(
                estimator=self.estimator,
                X_train=X_resampled,
                y_train=y_resampled
            ).get_results(
                X_test=X_test,
                y_test=y_test
            ))

            return scores
        
        def method_report(self):
            X_resampled, y_resampled = self.selection(
                X_train=self.X_train,
                y_train=self.y_train,
                method=self.method
            ).get_samples()

            report = self.calculate(
                estimator=self.estimator,
                X_train=X_resampled,
                y_train=y_resampled
            ).get_report(
                X_test=X_test,
                y_test=y_test
            )

            return report

        class selection:
            def __init__(self,X_train,y_train,method):
                self.X_train = X_train
                self.y_train = y_train
                self.method = method

            def get_samples(self):
                if self.method in ['1','2','3']:
                    method_dict = {
                        '1':RandomOverSampler,
                        '2':RandomUnderSampler,
                        '3':SMOTE
                    }

                    X_resampled, y_resampled = method_dict.setdefault(self.method)\
                        (random_state=1995).fit_resample(
                              X=self.X_train, 
                              y=self.y_train
                        )

                elif self.method == '4':
                    X_resampled, y_resampled = NearMiss().fit_resample(
                        X=self.X_train,
                        y=self.y_train
                    )

                else:
                    X_resampled = self.X_train
                    y_resampled = self.y_train

                return X_resampled, y_resampled

        class calculate:
            def __init__(self,estimator,X_train,y_train):
                self.estimator = estimator.fit(
                    X=X_train,
                    y=y_train
                )
            
            def get_results(self,X_test,y_test):
                ap_score = average_precision_score(
                    y_true=y_test,
                    y_score=self.estimator.predict_proba(X_test)[:,1]
                )

                f2_score = fbeta_score(
                    y_true=y_test,
                    y_pred=self.estimator.predict(X_test),
                    beta=2
                )

                pr_score = precision_score(
                    y_true=y_test,
                    y_pred=self.estimator.predict(X_test)
                )
                re_score = recall_score(
                    y_true=y_test,
                    y_pred=self.estimator.predict(X_test)
                )
                                    
                return [ap_score,f2_score,pr_score,re_score]
            
            def get_report(self,X_test,y_test):
                return classification_report(
                    y_true=y_test,
                    y_pred=self.estimator.predict(X_test)
                )

First of all, we will process cross validation score using method `imbalance_validation()` and display it in dataframe using training and validation data from `X_trainval` and `y_trainval`.

In [4]:
import pandas as pd

estimator = gb
sampling_score = imbalance_val_score(estimator).imbalance_validation(
    X_trainval=X_trainval,
    y_trainval=y_trainval
)

ap_scores, f2_scores, pr_scores, re_scores = [], [], [], []

for i,var in enumerate([ap_scores,f2_scores,pr_scores,re_scores]):
    for val in sampling_score[i]:
        var.append(val)

sampling_comparison = pd.DataFrame(data={
    ('Over Sampling','AP Score'):ap_scores[0],
    ('Over Sampling','F2 Score'):f2_scores[0],
    ('Over Sampling','Precision'):pr_scores[0],
    ('Over Sampling','Recall'):re_scores[0],
    ('Under Sampling','AP Score'):ap_scores[1],
    ('Under Sampling','F2 Score'):f2_scores[1],
    ('Under Sampling','Precision'):pr_scores[1],
    ('Under Sampling','Recall'):re_scores[1],
    ('SMOTE','AP Score'):ap_scores[2],
    ('SMOTE','F2 Score'):f2_scores[2],
    ('SMOTE','Precision'):pr_scores[2],
    ('SMOTE','Recall'):re_scores[2],
    ('Near Miss','AP Score'):ap_scores[3],
    ('Near Miss','F2 Score'):f2_scores[3],
    ('Near Miss','Precision'):pr_scores[3],
    ('Near Miss','Recall'):re_scores[3],
    ('Normal Sampling','AP Score'):ap_scores[4],
    ('Normal Sampling','F2 Score'):f2_scores[4],
    ('Normal Sampling','Precision'):pr_scores[4],
    ('Normal Sampling','Recall'):re_scores[4]
})

for function in [lambda x: x.mean(),lambda x: x.std()]:
    sampling_comparison = sampling_comparison.append(
        sampling_comparison.apply(
            func=function,
            axis=0
        ),
        ignore_index=True
    )
    
sampling_comparison = sampling_comparison.rename(index={
    0:'Fold 1',
    1:'Fold 2',
    2:'Fold 3',
    3:'Fold 4',
    4:'Fold 5',
    5:'Average',
    6:'STD'
})

sampling_comparison.apply(
    func=lambda x: round(
        number=x*100,
        ndigits=2
    )
).transpose()

Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Average  \
Over Sampling   AP Score    68.14   69.45   69.29   68.79   68.85    68.90   
                F2 Score    74.96   75.94   76.92   76.80   74.41    75.81   
                Precision   55.15   55.39   55.62   53.82   53.69    54.73   
                Recall      82.35   83.71   85.07   85.97   82.35    83.89   
Under Sampling  AP Score    66.70   68.75   66.51   67.94   65.59    67.10   
                F2 Score    75.34   76.12   75.22   75.32   75.71    75.54   
                Precision   52.38   52.20   52.09   51.65   53.28    52.32   
                Recall      84.62   85.97   84.62   85.07   84.62    84.98   
SMOTE           AP Score    67.60   69.56   69.52   69.17   70.59    69.29   
                F2 Score    72.15   73.15   71.90   72.68   74.00    72.78   
                Precision   55.84   55.17   54.23   55.59   56.73    55.51   
                Recall      77.83   79.64   78.28   78.73   80.09    78.91   
Near Miss       AP Score    28.66   21.66   35.49   24.23   26.97    27.40   
                F2 Score    60.29   55.29   61.09   57.27   57.76    58.34   
                Precision   33.33   28.15   39.15   30.24   33.05    32.79   
                Recall      75.57   72.85   71.04   73.76   71.04    72.85   
Normal Sampling AP Score    69.01   69.23   69.53   68.81   70.11    69.34   
                F2 Score    62.21   61.04   61.52   58.77   60.28    60.76   
                Precision   67.16   69.31   67.51   67.02   69.35    68.07   
                Recall      61.09   59.28   60.18   57.01   58.37    59.19   

                            STD  
Over Sampling   AP Score   0.46  
                F2 Score   0.99  
                Precision  0.81  
                Recall     1.45  
Under Sampling  AP Score   1.11  
                F2 Score   0.33  
                Precision  0.54  
                Recall     0.53  
SMOTE           AP Score   0.97  
                F2 Score   0.75  
                Precision  0.82  
                Recall     0.84  
Near Miss       AP Score   4.70  
                F2 Score   2.10  
                Precision  3.71  
                Recall     1.72  
Normal Sampling AP Score   0.45  
                F2 Score   1.18  
                Precision  1.04  
                Recall     1.41

It can be seen that the resampling method using SMOTE, over sampling, and under sampling has an Average Precision average result that is relatively equally good, with the best level of stability owned by `RandomOverSampling()` while the NearMiss produced the worst performance. On the other hand, based on the F2 Score measurement results, the over sampling and under sampling methods produce relatively equally good performance. For normal sampling, the Average Precision is quite good but has a much worse F2 Score because there is no resampling process to balance the amount of data for each target class.

F2 Score measurement can be used as an additional reference in assessing the performance of a resampling method in which metrics combine precision and recall values ​​with more emphasis on recall values. In the case of this study, the recall value indicates how many actual occurrences of churn customers can be predicted by the algorithm model. Therefore, it is very useful when the losses incurred due to false negatives or failure to predict cases of churn customers are relatively greater than the losses incurred from false positives or wrong predictions of churn customers.

##### **Imbalanced Testing**

Next, a testing process will be carried out using data test for all resampling methods tested. This can be done using method `imbalance_test_score()`.

In [5]:
testing_score = imbalance_val_score(estimator).imbalance_test_score(
    X_trainval=X_trainval,
    y_trainval=y_trainval,
    X_test=X_test,
    y_test=y_test
)

testing_comparison = pd.DataFrame(testing_score).apply(
    func=lambda x: round(
        number=x*100,
        ndigits=2
    )
).transpose()

testing_comparison.rename(
    index={
        0:'Over Sampling',
        1:'Under Sampling',
        2:'SMOTE',
        3:'Near Miss',
        4:'Normal Sampling'
    },
    columns={
        0:'AP Score',
        1:'F2 Score',
        2:'Precision',
        3:'Recall'
    }
)

,AP Score,F2 Score,Precision,Recall
Over Sampling,68.03,74.88,54.14,82.81
Under Sampling,68.22,74.96,52.86,83.71
SMOTE,69.90,72.56,55.24,78.73
Near Miss,27.70,58.80,33.20,72.85
Normal Sampling,69.44,60.59,66.50,59.28


It can be seen that SMOTE method is able to outperform other methods in terms of Average Precision measurement but has no better performance in terms of F2 Score measurement. This is similar to the results of the previous cross validation score. On the other hand, the over sampling and under sampling methods have relatively the same performance in terms of Average Precision and F2 Score. However, based on the results of the cross validation score, the over sampling method has a better level of stability in predicting the probability of each possible recall value even though in the testing process the scores are not better than the SMOTE and under sampling. Based on overall performance, the over sampling method will be used in the process of developing the necessary algorithm model according to the existing business problem.

Next, a classification report for each resampling method will be displayed as additional information using method `imbalance_classification_report()`.

In [6]:
reports = imbalance_val_score(estimator).imbalance_classification_report(
    X_trainval=X_trainval,
    y_trainval=y_trainval,
    X_test=X_test,
    y_test=y_test
)

for report,j in zip(reports,['Over Sampling','Under Sampling','SMOTE','Near Miss','Normal Sampling']):
    print(
        j.center(55,'='),
        '\n\n'+report+'\n'+('=').center(55,'='),
        '\n'
    )

=====================Over Sampling===================== 

              precision    recall  f1-score   support

           0       0.94      0.79      0.85       723
           1       0.54      0.83      0.65       221

    accuracy                           0.80       944
   macro avg       0.74      0.81      0.75       944
weighted avg       0.84      0.80      0.81       944


=====================Under Sampling==================== 

              precision    recall  f1-score   support

           0       0.94      0.77      0.85       723
           1       0.53      0.84      0.65       221

    accuracy                           0.79       944
   macro avg       0.73      0.80      0.75       944
weighted avg       0.84      0.79      0.80       944


=========================SMOTE========================= 

              precision    recall  f1-score   support

           0       0.93      0.80      0.86       723
           1       0.55      0.79      0.65       221

    ac